In [1]:
import json


In [2]:
import ee
ee.Authenticate()
ee.Initialize(project="ee-moethirizun97")

In [4]:
# Define a dictionary for legend and visualization
legend_dict = {
    "names": [
        "Water", "Trees", "Grass", "Flooded Vegetation", "Crops",
        "Scrub/Shrub", "Built Area", "Bare Ground", "Snow/Ice", "Clouds"
    ],
    "colors": [
        "#1A5BAB", "#358221", "#A7D282", "#87D19E", "#FFDB5C",
        "#EECFA8", "#ED022A", "#EDE9E4", "#F2FAFF", "#C8C8C8"
    ]
}

# Load the ESRI 2020 Land Cover data
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")




In [ ]:
# Retrieve information about the dataset
info = esri_lulc10.getInfo()

# Print available keys in the metadata dictionary
print(info.keys())


# Pretty print the first part of the metadata
print(json.dumps(info, indent=2))



{
  "type": "ImageCollection",
  "bands": [],
  "version": 1624668441802946,
  "id": "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m",
  "features": [
    {
      "type": "Image",
      "bands": [
        {
          "id": "b1",
          "data_type": {
            "type": "PixelType",
            "precision": "int",
            "min": 0,
            "max": 255
          },
          "dimensions": [
            27922,
            32025
          ],
          "crs": "EPSG:32701",
          "crs_transform": [
            10,
            0,
            365215.04551425716,
            0,
            -10,
            1381432.0991431829
          ]
        }
      ],
      "version": 1624644462274581,
      "id": "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m/01C_20200101-20210101",
      "properties": {
        "num_bands": 1,
        "id_no": "01C_20200101-20210101",
        "ysize": 32025,
        "system:footprint": {
          "type": "LinearRing",
         

In [5]:
# Create a map
import folium

# Function to add EE overlay to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters
vis_params = {
    'min': 1, 
    'max': 10, 
    'palette': legend_dict['colors']
}

# Create a folium map object.
my_map = folium.Map(location=[20.0, 0.0], zoom_start=3)

# Add the layer to the map
my_map.add_ee_layer(esri_lulc10.mosaic(), vis_params, 'ESRI LULC 10m')

# Display the map
display(my_map)

In [13]:
# Define the ESRI Land Cover dataset
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic()

# Define the bounding box for Australia
aoi = ee.Geometry.BBox(112.0, -44.0, 154.0, -10.0)

# Export the dataset as a GeoTIFF
task = ee.batch.Export.image.toDrive(
    image=esri_lulc10,
    description="ESRI_LandCover_Australia",
    region=aoi,
    scale=10,  # 10-meter resolution
    crs="EPSG:4326",  # Use European projection for consistency
    maxPixels=1e13
)
task.start()

print("Export started. Check your Google Drive for the file when complete.")

Export started. Check your Google Drive for the file when complete.


In [ ]:
import os
from osgeo import gdal

# Correct absolute path to the directory containing raster tiles
tiles_dir = r"C:\Users\moeth\OneDrive - Kyoto University\Side Studies\data science for esm\data-science-for-esm\data\lulc"

# Get a list of all .tif files in the directory
tile_files = [os.path.join(tiles_dir, f) for f in os.listdir(tiles_dir) if f.endswith(".tif")]

# Check if any .tif files are found
if not tile_files:
    raise FileNotFoundError("No .tif files found in the specified directory.")

# Define the output file path for the merged raster
output_file = os.path.join(tiles_dir, "australia_lulc.tif")

# Merge all the tiles into a single file
gdal.Warp(output_file, tile_files)

print(f"Merged file saved as: {output_file}")


In [ ]:
import folium
import rasterio
from folium.raster_layers import ImageOverlay

# Path to the merged GeoTIFF file for Australia's LULC
tif_file = r"C:\Users\moeth\OneDrive - Kyoto University\Side Studies\data science for esm\data-science-for-esm\data\lulc\australia_lulc.tif"

# Read the GeoTIFF file to get its bounds and data
with rasterio.open(tif_file) as src:
    bounds = src.bounds  # Geographic bounds of the raster
    data = src.read(1)  # Read the first band (assuming single-band raster)

# Convert the bounds to a format suitable for Folium (Lat/Lon)
bounds_latlon = [[bounds.bottom, bounds.left], [bounds.top, bounds.right]]

# Create a Folium map centered on Australia
m = folium.Map(
    location=[
        (bounds_latlon[0][0] + bounds_latlon[1][0]) / 2,
        (bounds_latlon[0][1] + bounds_latlon[1][1]) / 2,
    ],
    zoom_start=5,  # Adjust zoom level as needed
)

# Add the LULC raster as an overlay to the map
ImageOverlay(
    image=data,
    bounds=bounds_latlon,
    opacity=0.7,  # Adjust opacity for better visualization
    name="Australia LULC",
).add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m


In [7]:
file_path = "https://github.com/fneum/data-science-for-esm/raw/main/data-science-for-esm/U2018_CLC2018_V2020_20u1-PT.tif"

In [8]:
import rasterio
import numpy as np


# Open the GeoTIFF file
with rasterio.open(file_path) as dataset:
    # Read the first band
    data = dataset.read(1)
    
    # Mask out nodata values
    nodata = dataset.nodata
    if nodata is not None:
        data = np.ma.masked_equal(data, nodata)
    
    # Get unique land use/land cover types
    unique_types = np.unique(data.compressed())  # Use compressed to ignore masked values
    
    # Count the number of LULC types
    num_types = len(unique_types)
    
    # Display results
    print(f"Number of unique LULC types: {num_types}")
    print("Unique LULC types:", unique_types)


Number of unique LULC types: 42
Unique LULC types: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 37 38 39 40 41 42 43 44]


In [10]:
import atlite
import rasterio
import numpy as np
from osgeo import gdal
import geopandas as gpd
from rasterio.features import shapes

# Sample paths (replace with your actual data paths)
lulc_path = "https://github.com/fneum/data-science-for-esm/raw/main/data-science-for-esm/U2018_CLC2018_V2020_20u1-PT.tif"  # Replace with your LULC GeoTIFF
dem_path = "http://gis.ciimar.up.pt/gisData/dems/dem_srtm_pt_1sec.tif"    # Replace with your DEM GeoTIFF

# Step 1: Prepare the Cutout
cutout = atlite.Cutout(
    path="sample_cutout.nc",
    module="era5",
    x=slice(-10, 30),  # Longitude range (replace as needed)
    y=slice(35, 70),   # Latitude range (replace as needed)
    time=slice("2020-01-01", "2020-12-31"),  # Time range
)
cutout.prepare()

# Step 2: Process LULC Data
with rasterio.open(lulc_path) as lulc_src:
    lulc_data = lulc_src.read(1)
    lulc_transform = lulc_src.transform

# Define exclusion criteria for LULC (e.g., exclude forests, water bodies)
excluded_lulc_classes = [3, 5]  # Modify these based on your LULC legend
lulc_mask = np.isin(lulc_data, excluded_lulc_classes)

# Step 3: Process DEM and Calculate Slope
gdal.DEMProcessing("slope.tif", dem_path, "slope", computeEdges=True)
with rasterio.open("slope.tif") as slope_src:
    slope_data = slope_src.read(1)

# Define exclusion criteria for slope (e.g., slopes > 10 degrees)
slope_threshold = 10
slope_mask = slope_data > slope_threshold

# Step 4: Combine Exclusion Masks
combined_exclusion_mask = lulc_mask | slope_mask

# Step 5: Create Exclusion Container
shapes_gen = shapes(combined_exclusion_mask.astype(np.uint8), transform=lulc_transform)
polygons = [geom[0] for geom in shapes_gen if geom[1] == 1]
exclusion_gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")  # Replace with your CRS

# Add exclusion data to atlite
exclusion_container = atlite.ExclusionContainer(crs=cutout.crs)
exclusion_container.add_geometry(exclusion_gdf)

# Apply exclusions to cutout
cutout.exclude(exclusion_container)

# Save the modified cutout
cutout.to_netcdf("sample_cutout_with_exclusions.nc")
print("Exclusion container applied and saved.")


Exception: Missing/incomplete configuration file: C:\Users\moeth/.cdsapirc